In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:

from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
#import warnings
#warnings.filterwarnings("ignore", category=FutureWarning)


In [3]:
train_input="dataset/topic/train.csv"
test_input="dataset/topic/valid.csv"
topic_input="dataset/topic/topics.txt"
voc_input="dataset/topic/vocabulary.txt"

In [4]:
def load_class():
    CLASSES = set()
    with open(topic_input, 'r', encoding="utf-8") as f:
        for line in f:
            CLASSES.add(str(line[:-1]))
    return list(CLASSES)

CLASSES = load_class()

In [5]:
def load_voc():
    with open(voc_input, "r", encoding="utf-8") as file:
        voc = ast.literal_eval(file.readline())
    return voc


In [6]:
def topic_to_string(nb):
    return CLASSES[nb]

In [7]:
voc=load_voc()
voc = dict(map(reversed, voc.items()))

def text_to_string(text):
    text=text[1:-1]
    sentence = ""
    for word in text.split(', '):
        sentence+=voc[int(word)]
        sentence+=" "
    return sentence

In [8]:
train = pd.read_csv(train_input,index_col=0,nrows=1000)
train.shape


(1000, 2)

In [9]:
test = pd.read_csv(test_input,index_col=0,nrows=100)
test.shape

(100, 2)

In [10]:
train['topic_name']=train['topic'].apply(topic_to_string)
train['text']=train['text'].apply(text_to_string)
test['topic_name']=test['topic'].apply(topic_to_string)
test['text']=test['text'].apply(text_to_string)

In [11]:
test.head(100)

,topic,text,topic_name
5,178,menu them trac ecrit région grand ten club fab...,Fonderie d'autres métaux non ferreux
14,93,laurent bendavid général logist franc depuis v...,Travaux d'installation d'équipements thermique...
18,61,part découvr offre marseil justic ordon expuls...,Construction de navires et de structures flott...
20,648,richardson prend contrôl societ magelec distri...,Conseil en systèmes et logiciels informatiques
36,11,publi gwénaëll barzic paris reuter vincent bol...,Fabrication de machines pour l'extraction ou l...
...,...,...,...
1030,232,résultat opérationnel cour plus haut histor de...,Fonderie d'acier
1041,105,consulterl journal projet mégacomplex europaci...,Commerce de détail de produits surgelés
1054,254,restructur financi marqu a finalis mai ikk pas...,Raffinage du pétrole
1066,346,édit sarth sarth industr cédric menuet septemb...,Autres services d'information n.c.a.


In [12]:
train.iloc[4,1]


'laurent guillaum publi novembr villéo logiv fusion don naissanc habel acteur premi plan log social pres log ger bourgogn démarch phas anticip loi elan cibl not consolid tissu organ log social communiqu press group action log depuis début nouveau group action log réform modernis officiel constitu cet réform initi a about instaur deux principal pôl activ pôl servic action log servic pôl immobili action log immobili met oeuvr strateg immobili group cet approch ambiti repos strateg structur réseau filial immobili ambit group dot organis cohérent rationnel efficient lien bassin emploi spécif territoir action log souhait marqu ancrag territorial fort économ social solidair motiv projet fusion villéo logiv deux structur taill similair interven territoir commun ains context réform mouv action log projet loi elan conseil administr deux structur valid position direct général partag part mar cet approch a perm affin réflexion axe possibl développ trajectoir commun cour septembr proposit rapproch

In [13]:
# Remove missing values (NaN)
#train = train[pd.notnull(train['text'])]
#train.shape


#pd.DataFrame(train.topic.unique()).values



In [14]:
tfidf = TfidfVectorizer(sublinear_tf=True,
                        ngram_range=(1, 2))



In [15]:
x_train, x_test, y_train, y_test,indices_train,indices_test = train_test_split(tfidf.fit_transform(train.text).toarray(), 
                                                               train['topic'], 
                                                               train.index, test_size=0.25, 
                                                               random_state=1)

In [23]:
y_train.shape

(750,)

In [17]:
model = LinearSVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [31]:
print(y_pred)
print(y_test)
# Classification report
print('\t\t\t\tCLASSIFICATIION METRICS\n')
print(metrics.classification_report(y_test, y_pred, 
                                    target_names= train['topic_name'].unique()))


[ 93 444  93  93  93  17  93  93  93  93  93 629  93  93  93 144 483  93
  93 586  93  93  93  93 606 230  93  93  93  17  93  93 167 230  52 376
  93  11 652 383  93  93  93  93  21  93 650  93  11 230 527  38  93  38
  59  93  93  93  93 527 136 337  93  93  93  93 652  93  89  93  93  17
  93 199  93  93  93  93  93  93  93 230  11  93  93 230  93 232  38 230
 444 141  38  11  89 232 527  93  93  93 527 166 292  93  38  93  93 432
 337 343  93  93  93  93  93 153  93 370  93  93  93  93  93  93 230 383
  80 167  93 527  93  93  93  93 230 230  93 230 527  93 138 586  93  93
  11  87  87  93  90  90 199  93  93 232 505  42 230  38 334 150  93  11
  93 464   0  17  93  93  93  93  87  93  93 444  93  93 232 230 527  11
 383  93 551 232  38  93  93  93  11 586 527  93  11 232 334  93  93  11
  93 622  93 230  93  11  93 230  93 263 528 144  93  93  93  93 527 650
  93  93  11 652 109 625  93  11 527 232  42 167  93  87  93 253  93  59
 292 464 527  93  93  93 199  93  93  93  93 471 62

ValueError: Number of classes, 151, does not match size of target_names, 281. Try specifying the labels parameter